In [1]:
import requests, json
from pprint import pprint

In [2]:
# base url for all FPL API endpoints
base_url = 'https://fantasy.premierleague.com/api/'

# get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 9012937}


In [3]:
# get player data from 'elements' field
players = r['elements']

# show data for first player
pprint(players[0])

{'assists': 0,
 'bonus': 0,
 'bps': 48,
 'chance_of_playing_next_round': 100,
 'chance_of_playing_this_round': 100,
 'clean_sheets': 0,
 'code': 80201,
 'corners_and_indirect_freekicks_order': None,
 'corners_and_indirect_freekicks_text': '',
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': -5,
 'cost_change_start_fall': 5,
 'creativity': '0.0',
 'creativity_rank': 598,
 'creativity_rank_type': 69,
 'direct_freekicks_order': None,
 'direct_freekicks_text': '',
 'dreamteam_count': 0,
 'element_type': 1,
 'ep_next': '1.0',
 'ep_this': '0.0',
 'event_points': 0,
 'first_name': 'Bernd',
 'form': '0.0',
 'goals_conceded': 9,
 'goals_scored': 0,
 'ict_index': '7.9',
 'ict_index_rank': 419,
 'ict_index_rank_type': 30,
 'id': 1,
 'in_dreamteam': False,
 'influence': '79.0',
 'influence_rank': 363,
 'influence_rank_type': 29,
 'minutes': 270,
 'news': '',
 'news_added': '2022-02-11T08:00:15.144286Z',
 'now_cost': 45,
 'own_goals': 0,
 'penalties_missed': 0,
 'penalti

In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [5]:
# create players dataframe
players = pd.json_normalize(r['elements'])

# show some information about first five players
players[['id', 'web_name', 'team', 'element_type']].head()

,id,web_name,team,element_type
0,1,Leno,1,1
1,2,Rúnarsson,1,1
2,3,Willian,1,3
3,4,Aubameyang,1,4
4,5,Cédric,1,2


In [6]:
# create teams dataframe
teams = pd.json_normalize(r['teams'])

teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1210,1270,1150,1210,1190,1220,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1130,1160,1140,1110,1090,1090,2
2,94,0,None,3,0,Brentford,0,0,0,BRE,2,None,False,0,1060,1070,1120,1150,1080,1120,130
3,36,0,None,4,0,Brighton,0,0,0,BHA,3,None,False,0,1130,1130,1160,1160,1100,1120,131
4,90,0,None,5,0,Burnley,0,0,0,BUR,2,None,False,0,1060,1050,1080,1130,1060,1100,43


In [7]:
# get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],242
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],296
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],93


In [8]:
# join players to teams
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

# show joined result
df[['first_name', 'second_name', 'name']].head()

,first_name,second_name,name
0,Bernd,Leno,Arsenal
1,Rúnar Alex,Rúnarsson,Arsenal
2,Willian,Borges Da Silva,Arsenal
3,Pierre-Emerick,Aubameyang,Arsenal
4,Cédric,Soares,Arsenal


In [9]:
# join player positions
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id'
)

# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)

# show result
new_df = df[['first_name', 'second_name', 'team_name', 'position_name', 'now_cost', 'total_points']]

In [11]:
new_df.sort_values(by = ['total_points'], ascending=False)

,first_name,second_name,team_name,position_name,now_cost,total_points
240,Mohamed,Salah,Liverpool,Midfielder,131,209
591,Trent,Alexander-Arnold,Liverpool,Defender,85,162
355,Jarrod,Bowen,West Ham,Midfielder,71,149
605,João Pedro Cavaco,Cancelo,Man City,Defender,71,148
319,Heung-Min,Son,Spurs,Midfielder,108,137
...,...,...,...,...,...,...
546,Jarosław,Jach,Crystal Palace,Defender,43,0
263,Romeo,Lavia,Man City,Midfielder,44,0
262,Tommy,Doyle,Man City,Midfielder,45,0
261,Samuel,Edozie,Man City,Midfielder,47,0


In [13]:
url = 'https://fantasy.premierleague.com/api/element-summary/4/'

# get data from bootstrap-static endpoint
r = requests.get(url).json()

# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{'fixtures': [...], 'history': [...], 'history_past': [...]}


In [17]:
# get player data from 'elements' field
player_hist = pd.json_normalize(r['history_past'])
player_hist

,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index
0,2017/18,54694,105,108,87,1056,10,4,4,15,0,0,1,0,0,0,12,354,484.0,207.1,580.0,127.1
1,2018/19,54694,110,108,205,2722,22,7,7,42,0,0,1,0,0,0,32,756,984.4,484.6,1429.0,289.6
2,2019/20,54694,110,109,205,3136,22,5,10,44,0,0,0,3,1,0,37,807,1006.0,479.6,1369.0,285.2
3,2020/21,54694,120,113,131,2330,10,3,10,26,1,0,0,2,0,0,11,358,493.8,361.5,823.0,167.4


In [21]:
def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    
    return df


# show player #1's gameweek history
get_season_history(1)[
    [
        'season_name',
        'total_points',
        'end_cost'
    ]
].head(10)

,season_name,total_points,minutes,goals_scored,assists,end_cost,total_points
0,2018/19,106,2835,0,0,49,106
1,2019/20,114,2649,0,0,48,114
2,2020/21,131,3131,0,0,50,131


In [19]:
# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type']
]

# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

players.head()

,id_player,first_name,second_name,web_name,name,singular_name_short
0,1,Bernd,Leno,Leno,Arsenal,GKP
1,2,Rúnar Alex,Rúnarsson,Rúnarsson,Arsenal,GKP
2,532,Karl,Hein,Hein,Arsenal,GKP
3,559,Aaron,Ramsdale,Ramsdale,Arsenal,GKP
4,572,Arthur,Okonkwo,Okonkwo,Arsenal,GKP


In [22]:
from tqdm.auto import tqdm
tqdm.pandas()

/Users/rachelchen/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
# get seasonhistories for each player
points = players['id_player'].progress_apply(get_season_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

# join web_name
points = players[['id_player', 'web_name']].merge(
    points,
    left_on='id_player',
    right_on='element'
)